In [2]:
import pickle

In [1]:
import numpy as np
from datasets import load_dataset

dataset = load_dataset("kmfoda/booksum")

/home/kims90/virtual_envs/mobile_prj/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (/home/kims90/.cache/huggingface/datasets/kmfoda___csv/kmfoda--booksum-025141c210e07407/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 3/3 [00:00<00:00, 191.02it/s]


In [2]:
from summarizer import Summarizer
body = 'Text body that you want to summarize with BERT'
body2 = 'Something else you want to summarize with BERT'
model = Summarizer()
model(body)
model(body2)


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


'Something else you want to summarize with BERT'

In [3]:
model(dataset["test"]["chapter"][0], ratio=0.3)

/home/kims90/virtual_envs/mobile_prj/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


'All states, all powers, that have held and hold rule over men have been\nand are either republics or principalities. Principalities are either hereditary, in which the family has been long\nestablished; or they are new. The new are either entirely new, as was Milan to Francesco Sforza, or\nthey are, as it were, members annexed to the hereditary state of the\nprince who has acquired them, as was the kingdom of Naples to that of\nthe King of Spain. Such dominions thus acquired are either accustomed to live under a\nprince, or to live in freedom; and are acquired either by the arms of\nthe prince himself, or of others, or else by fortune or by ability. We have in Italy, for example, the Duke of Ferrara, who could not have\nwithstood the attacks of the Venetians in \'84, nor those of Pope Julius\nin \'10, unless he had been long established in his dominions. But the difficulties occur in a new principality. This follows also on another natural and common\nnecessity, which always causes a 

In [4]:
from rouge import Rouge

hypothesis = model(dataset["test"]["chapter"][0])

reference = dataset["test"]["summary_text"][0]

rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)

/home/kims90/virtual_envs/mobile_prj/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [5]:
scores

[{'rouge-1': {'r': 0.23699421965317918,
   'p': 0.3504273504273504,
   'f': 0.2827586158760999},
  'rouge-2': {'r': 0.04655172413793104,
   'p': 0.07180851063829788,
   'f': 0.05648535087621056},
  'rouge-l': {'r': 0.1936416184971098,
   'p': 0.2863247863247863,
   'f': 0.2310344779450655}}]

In [6]:
scores[0]['rouge-1']['r']

0.23699421965317918

In [7]:
len(dataset["test"])

1431

In [8]:
scores_dict = {'rouge-1': {'r': np.zeros((len(dataset["test"], ))), 'p': np.zeros((len(dataset["test"], ))), 'f': np.zeros((len(dataset["test"], )))}, 
               'rouge-2': {'r': np.zeros((len(dataset["test"], ))), 'p': np.zeros((len(dataset["test"], ))), 'f': np.zeros((len(dataset["test"], )))}, 
               'rouge-l': {'r': np.zeros((len(dataset["test"], ))), 'p': np.zeros((len(dataset["test"], ))), 'f': np.zeros((len(dataset["test"], )))}, }


In [9]:
from tqdm import tqdm

In [10]:
import torch
assert torch.cuda.is_available()

In [11]:
for i in tqdm(range(len(dataset["test"]))):
    hypothesis = model(dataset["test"]["chapter"][i])
    reference = dataset["test"]["summary_text"][i]
    scores = rouge.get_scores(hypothesis, reference)

    for j in scores_dict.keys():
        for k in scores_dict[j].keys():
            scores_dict[j][k][i] = scores[0][j][k]


  0%|          | 0/1431 [00:00<?, ?it/s]/home/kims90/virtual_envs/mobile_prj/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
  0%|          | 1/1431 [00:01<31:37,  1.33s/it]/home/kims90/virtual_envs/mobile_prj/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
  0%|          | 2/1431 [00:04<52:39,  2.21s/it]/home/kims90/virtual_envs/mobile_prj/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
  0%|          | 3/1431 [00:05<41:38,  1.75s/it]/home/kims90/virtual_envs/mobile_prj/lib/p

KeyboardInterrupt: 

In [15]:
print(np.mean(scores_dict["rouge-1"]['r'][:1180]))
print(np.mean(scores_dict["rouge-2"]['r'][:1180]))
print(np.mean(scores_dict["rouge-l"]['r'][:1180]))

0.3309874517162135
0.05724912709832521
0.3013532391075907


In [ ]:
print(sum(scores_dict["rouge-1"]['r'])/len(scores_dict["rouge-1"]['r']))
print(sum(scores_dict["rouge-2"]['r'])/len(scores_dict["rouge-2"]['r']))
print(sum(scores_dict["rouge-l"]['r'])/len(scores_dict["rouge-l"]['r']))